## 安裝套件

In [1]:
!pip install requests

In [2]:
!pip install feedparser

In [3]:
!pip install lxml

In [4]:
!pip install jieba

### 下載繁體字的辭典檔

In [5]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O jieba_data/dict.txt.big

'wget' 不是內部或外部命令、可執行的程式或批次檔。


In [6]:
!pip install wordcloud

## 解析 RSS

In [7]:
import feedparser

In [8]:
rss_url = "https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money"

In [9]:
newsFeed = feedparser.parse(rss_url)
newsFeed

{'bozo': True,
 'entries': [{'title': '英媒：俄羅斯密件顯示蒲亭2016年下令助川普勝選',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money',
    'value': '英媒：俄羅斯密件顯示蒲亭2016年下令助川普勝選'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://money.udn.com/money/story/10511/5605400'}],
   'link': 'https://money.udn.com/money/story/10511/5605400',
   'published': 'Fri, 16 Jul 2021 10:12:21 +0800',
   'published_parsed': time.struct_time(tm_year=2021, tm_mon=7, tm_mday=16, tm_hour=2, tm_min=12, tm_sec=21, tm_wday=4, tm_yday=197, tm_isdst=0),
   'summary': '英國「衛報」（The Guardian）引用據稱是俄羅斯克里姆林宮外洩的文件報導，俄國總統蒲亭曾在國安會議上批准特務行動...',
   'summary_detail': {'type': 'text/html',
    'language': None,
    'base': 'https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money',
    'value': '英國「衛報」（The Guardian）引用據稱是俄羅斯克里姆林宮外洩的文件報導，俄國總統蒲亭曾在國安會議上批准特務行動...'},
   'id': 'https://money.udn.com/money/story/10511/5605400',
   'gu

### 查看每個 entry 的 title, url 

In [10]:
i = 1
for e in newsFeed['entries']:
    title = e['title']
    link_url = e['links'][0]['href']
    print("%s, %s, %s"%(i, title, link_url))
    i = i + 1

1, 英媒：俄羅斯密件顯示蒲亭2016年下令助川普勝選, https://money.udn.com/money/story/10511/5605400
2, 葉倫無意重啟美中經濟對話 前朝做法延續, https://money.udn.com/money/story/10511/5604903
3, 為戰勝中國 美國防部將投資15億美元發展AI軍備, https://money.udn.com/money/story/10511/5604668
4, 美軍機來台 中國國防部：將採取一切必要措施, https://money.udn.com/money/story/10511/5604101
5, 董承濂創造奇幻時刻 疫情紛擾下找回內心寧靜, https://money.udn.com/money/story/10511/5604060
6, 制裁中國又一舉 美參院通過禁止進口新疆產品, https://money.udn.com/money/story/10511/5602984
7, 現代紅色電話？拜登政府擬建「美中熱線」、供緊急通話, https://money.udn.com/money/story/10511/5602610
8, 川規拜隨 葉倫無意重啟美中高層經濟對話, https://money.udn.com/money/story/10511/5602551
9, 美拍板537億元補助美企 汰換華為與中興通訊設備, https://money.udn.com/money/story/10511/5600220
10, 拜登提名前國防部官員 掌美中科技戰關鍵職位, https://money.udn.com/money/story/10511/5600124
11, 葉倫倡美歐統一戰線對抗中國政經惡行 北京不滿, https://money.udn.com/money/story/10511/5599739
12, 美稱陸在黑山推「債務陷阱外交」 陸外交部批居心不良, https://money.udn.com/money/story/10511/5599138
13, 拜登政府擬在亞太區推數位貿易協定  制衡中國, https://money.udn.com/money/story/10511/5597474
14, 美國想切斷中國企業金脈 北京助

### 解析html可以參考 https://lxml.de/lxmlhtml.html

## 開始取得文章

### 載入 library

In [11]:
import requests
from lxml import etree
from io import StringIO
import jieba
import pandas as pd
import time
import random

### 使用繁體辭典

In [12]:
#指定辭典檔
jieba.set_dictionary('jieba_data/dict.txt.big')

### 設定 request header

In [13]:
#設定 request header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
my_headers = {
    'User-Agent': user_agent,
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "accept-encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "cache-control": "no-cache",
    "Accept-Charset": "UTF8,utf-8;q=0.7,*;q=0.7"
}

### 先爬文

In [ ]:
#分解動作

#爬文
each_article_text_list = []
for e in newsFeed['entries']:
    url = e['links'][0]['href']
    print(url)
    r = requests.get(url, headers = my_headers)
    if r.status_code == 200:
        parse_tree = etree.parse(StringIO(r.text), etree.HTMLParser())
        article_elements = parse_tree.xpath('//*[@id="article_body"]//p')
        for a_part in article_elements:
            if type(a_part.text) is str:
                each_article_text_list.append(a_part.text.strip())
        sleep_time = random.randint(3,10)
        print("sleep time: %s sec"%(sleep_time))
        time.sleep(sleep_time)
    all_article_text = ''.join(each_article_text_list)
all_article_text

https://money.udn.com/money/story/10511/5605400
sleep time: 8 sec
https://money.udn.com/money/story/10511/5604903
sleep time: 10 sec
https://money.udn.com/money/story/10511/5604668
sleep time: 10 sec
https://money.udn.com/money/story/10511/5604101
sleep time: 4 sec
https://money.udn.com/money/story/10511/5604060
sleep time: 5 sec
https://money.udn.com/money/story/10511/5602984
sleep time: 8 sec
https://money.udn.com/money/story/10511/5602610
sleep time: 6 sec
https://money.udn.com/money/story/10511/5602551
sleep time: 5 sec
https://money.udn.com/money/story/10511/5600220
sleep time: 6 sec
https://money.udn.com/money/story/10511/5600124
sleep time: 6 sec
https://money.udn.com/money/story/10511/5599739
sleep time: 4 sec
https://money.udn.com/money/story/10511/5599138
sleep time: 9 sec


### 做分詞

In [ ]:
#分詞
seg_words_list = jieba.lcut(all_article_text)
seg_words_list

In [ ]:
!ls jieba_data

### 準備 stop word

In [ ]:
#stop word
with open(file='jieba_data/stop_words.txt', mode='r', encoding='utf-8') as file:
    stop_words = file.read().split('\n')
stop_words

### 準備一個利用stop word過濾後的分詞結果

In [ ]:
#分詞
seg_stop_words_list = []
seg_words_list = jieba.lcut(all_article_text)
for term in seg_words_list:
    if term not in stop_words:
        seg_stop_words_list.append(term)
seg_stop_words_list

In [ ]:
#合併動作 XD
each_article_text_list = []
seg_words_list = []
for e in newsFeed['entries']:
    url = e['links'][0]['href']
    print(url)
    r = requests.get(url, headers = my_headers)
    if r.status_code == 200:
        parse_tree = etree.parse(StringIO(r.text), etree.HTMLParser())
        article_elements = parse_tree.xpath('//article//p')
        for a_part in article_elements:
            if type(a_part.text) is str:
                term = a_part.text.strip()
                if term not in stop_words:
                    each_article_text_list.append(term)
        all_article_text += ''.join(each_article_text_list)
        sleep_time = random.randint(3,10)
        print("sleep time: %s sec"%(sleep_time))
        time.sleep(sleep_time)
seg_words_list = jieba.lcut(all_article_text)
seg_df = pd.DataFrame(seg_words_list, columns=['seg_word'])
print(seg_df)

### 載入繪圖相關的 library

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
print(matplotlib.matplotlib_fname())

### 安裝中文字型

In [ ]:
!ls fonts/*.ttf

In [ ]:
#!cat /opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc

#### 字型檔案是放在 /opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf 我們先列出來看一下

In [ ]:
!ls /opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf

#### 將中文字型檔案放到字型收容的路徑

In [ ]:
!cp fonts/*.ttf /opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf 

#### 檢查一下結果

In [ ]:
!ls /opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf

#### 清除 matplotlib 快取

In [ ]:
print(matplotlib.get_configdir())

In [ ]:
!ls -la /home/jovyan/.config/matplotlib

In [ ]:
print(matplotlib.get_cachedir())

In [ ]:
!rm -rf /home/jovyan/.cache/matplotlib

In [ ]:
#用來顯示中文
plt.rcParams['font.sans-serif']=['SimHei']
#用來顯示負號
plt.rcParams['axes.unicode_minus']=False

#### 其他 FAQ 可以參考 https://matplotlib.org/3.1.1/faq/troubleshooting_faq.html

### 繪製文字雲 ( wordcloud.generate )

In [ ]:
# wordcloud.generate 參數是吃文字內容，以空白為文字區隔
seg_words = ' '.join(seg_words_list)
#seg_words

In [ ]:
#不同字型
wordcloud = WordCloud(font_path='fonts/SimHei.ttf').generate(seg_words)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#不同字型
wordcloud = WordCloud(font_path='fonts/TaipeiSansTCBeta-Regular.ttf').generate(seg_words)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### 繪製文字雲 ( wordcloud.generate_from_frequencies )

In [ ]:
# https://docs.python.org/zh-tw/3/library/collections.html
# 準備一個沒有經過stop word 處理的 counter
from collections import Counter
seg_counter = Counter(seg_words_list)
seg_counter

In [ ]:
# 準備一個有經過stop word 處理的 counter
from collections import Counter
seg_stop_counter = Counter(seg_stop_words_list)
#seg_stop_counter

In [ ]:
#沒有處理stop words
wordcloud = WordCloud(font_path='fonts/TaipeiSansTCBeta-Regular.ttf').generate_from_frequencies(seg_counter)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#有處理stop words
wordcloud = WordCloud(font_path='fonts/TaipeiSansTCBeta-Regular.ttf').generate_from_frequencies(seg_stop_counter)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### [其他] 用dataframe 處理數據

In [ ]:
seg_df = pd.DataFrame(seg_words_list, columns=['seg_word'])
seg_df

In [ ]:
seg_df['count'] = 1
seg_freq_df = seg_df.groupby('seg_word').sum()
seg_freq_df

In [ ]:
seg_df.groupby('seg_word').sum().plot.bar()
plt.xticks(rotation=50)
plt.show()

### [其他] parse html 小筆記

In [ ]:
import requests
from lxml import etree
import jieba
from io import StringIO
import pandas as pd

for e in newsFeed['entries']:
    url = e['links'][0]['href']
    print(url)
    r = requests.get(url)
    #print(r.status_code == 200)
    if r.status_code == 200:
        html_tree = etree.HTML(r.text)
        print(html_tree)
        html_ele = html_tree.xpath('//article//p')
        all_ele = [ (ele.text.strip()) for ele in html_ele if type(ele.text) is str ]
        all_ele_string = ''.join(all_ele)
        seg_words = jieba.lcut(all_ele_string)
        seg_df = pd.DataFrame(seg_words, columns=['seg_word'])
        seg_df['count'] = 1
        print(seg_df)